# <b><font color="#FF6633">LBP + Alex</font></b>

## 模型定义

In [1]:
from tensorflow.keras.layers import Conv2D,MaxPooling2D,BatchNormalization,ReLU,Flatten,Dense,Dropout
from tensorflow.keras.models import Sequential
import tensorflow.keras.optimizers as optimizers

MODEL_LOSS = 'categorical_crossentropy'
MODEL_METRIC = 'categorical_accuracy'

def InitialiazeModel(lr):
    """
    head_only:选择是否只训练顶端（即自定义的全连接层）
    weights:选择是否从外部导入权重
    model:模型名称
    lr:学习率：默认为0.001
    """
    model = Sequential()
    in_shape=(512, 512, 1)
    model.add(Conv2D(32, (5, 5),kernel_initializer='he_uniform', strides=(3,3),padding='valid', input_shape=in_shape))
    model.add(BatchNormalization(momentum=0.9)) # 根据batch_size修改
    model.add(ReLU())
    model.add(MaxPooling2D((3, 3),strides=1,padding='same'))
    # repeat
    model.add(Conv2D(32, (5, 5),kernel_initializer='he_uniform', strides=(3,3),padding='valid'))
    model.add(BatchNormalization(momentum=0.9)) # 根据batch_size修改
    model.add(ReLU())
    model.add(MaxPooling2D((3, 3),strides=1,padding='same'))
    # repeat
    model.add(Conv2D(32, (5, 5),kernel_initializer='he_uniform', strides=(3,3),padding='valid'))
    model.add(BatchNormalization(momentum=0.9)) # 根据batch_size修改
    model.add(ReLU())
    model.add(MaxPooling2D((3, 3),strides=1,padding='same'))
    
    model.add(Flatten())
    model.add(Dense(1024, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dropout(0.2))
    model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dropout(0.2))
    model.add(Dense(10, activation='softmax'))
    
    # ========================= 优化器 =======================================
    MODEL_OPTIMIZER = optimizers.SGD(lr=lr, momentum=0.9, nesterov=True)
    # MODEL_OPTIMIZER = optimizers.Adam()
    # 编译模型
    model.compile(loss=MODEL_LOSS, optimizer=MODEL_OPTIMIZER, metrics=[MODEL_METRIC])
    # model.summary()
    return model

## 目录

In [3]:
import os
category = os.listdir("datasets\\Train")
category.remove('.ipynb_checkpoints')
print(category)

['Apple_iPhone6Plus', 'Canon_PowerShotA640', 'Huawei_P9', 'Lenovo_P70A', 'Microsoft_Lumia640LTE', 'Nikon_D70s', 'OnePlus_A3003', 'Samsung_GalaxyS5', 'Sony_DSC-W170', 'Xiaomi_RedmiNote3']


## 生成器

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from skimage.feature import local_binary_pattern
from tensorflow.image import rgb_to_grayscale
import numpy as np
import sys

def LBP_gen(directory,classes):
    # 包引入
    data_gen = ImageDataGenerator()
    train_it = data_gen.flow_from_directory(directory=directory,target_size=(256,256),
                                           classes=category,class_mode= "categorical",
                                            batch_size=BATCH_SIZE)
    while True:
        X,y = next(train_it)
        g = rgb_to_grayscale(X)
        g = g.numpy() 
        shape = g.shape
        for i in range(shape[0]):
            g[i,:256, :256,0] = local_binary_pattern(g[i,:256, :256,0], 8, 1)
                
        yield(g[:,:256,:256,:],y)   
        

## 训练

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping,TensorBoard,ReduceLROnPlateau
from math import ceil

train_path = "datasets\\Train"
val_path = "datasets\\Val_main"
BATCH_SIZE = 32
train_sample_num = 30000
val_sample_num = 1555

# 训练
model = InitialiazeModel(lr=0.01)
weights_path_name = "model_weight\\LBP\\lbp{epoch:02d}.hdf5" 
callbacks = [ModelCheckpoint(weights_path_name, monitor='val_loss', save_best_only=True, verbose=0,
                                             save_weights_only=True),
             EarlyStopping(monitor='val_loss', patience=5, verbose=0.001),
             TensorBoard(log_dir='train_log\\Alex',update_freq='epoch'),
             ReduceLROnPlateau(factor=0.4,
                               patience=1, 
                              min_lr=0.001)]
history1 = model.fit_generator(generator = LBP_gen(train_path,category),
                    validation_data = LBP_gen(val_path,category),
                    epochs = 100,
                    steps_per_epoch=ceil(train_sample_num/ BATCH_SIZE),
                    validation_steps=ceil(val_sample_num/ BATCH_SIZE),
                   max_queue_size=20,
                   callbacks=callbacks,
                   verbose = 1)

Instructions for updating:
Please use Model.fit, which supports generators.
Found 30150 images belonging to 10 classes.
Epoch 1/100
  1/938 [..............................] - ETA: 0s - loss: 4.0834 - categorical_accuracy: 0.1562WARNING:tensorflow:From F:\anaconda\envs\tensorflow2.3.1\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
938/938 [==============================] - 1446s 2s/step - loss: 1.9370 - categorical_accuracy: 0.3330 - val_loss: 2.9326 - val_categorical_accuracy: 0.1891
Epoch 2/100
938/938 [==============================] - 1428s 2s/step - loss: 1.1198 - categorical_accuracy: 0.6192 - val_loss: 1.5254 - val_categorical_accuracy: 0.4617
Epoch 3/100
938/938 [==============================] - 1413s 2s/step - loss: 0.8997 - categorical_accuracy: 0.6941 - val_loss: 1.2727 - val_categorical_accu

## 版本二：原汁原味的模型

In [2]:
from tensorflow.keras.layers import Conv2D,MaxPooling2D,BatchNormalization,ReLU,Flatten,Dense,Dropout
from tensorflow.keras.models import Sequential
import tensorflow.keras.optimizers as optimizers

MODEL_LOSS = 'categorical_crossentropy'
MODEL_METRIC = 'categorical_accuracy'

def InitialiazeModel(lr):
    """
    head_only:选择是否只训练顶端（即自定义的全连接层）
    weights:选择是否从外部导入权重
    model:模型名称
    lr:学习率：默认为0.001
    """
    model = Sequential()
    in_shape=(256, 256, 1)
    model.add(Conv2D(64, (3, 3),kernel_initializer='he_uniform', strides=(2,2),padding='valid', input_shape=in_shape))
    model.add(BatchNormalization(momentum=0.95)) # 根据batch_size修改
    model.add(ReLU())
    model.add(MaxPooling2D((3, 3),strides=1,padding='same'))
    # repeat
    model.add(Conv2D(64, (3, 3),kernel_initializer='he_uniform', strides=(2,2),padding='valid'))
    model.add(BatchNormalization(momentum=0.95)) # 根据batch_size修改
    model.add(ReLU())
    model.add(MaxPooling2D((3, 3),strides=1,padding='same'))
    # repeat
    model.add(Conv2D(32, (3, 3),kernel_initializer='he_uniform', strides=(2,2),padding='valid'))
    model.add(BatchNormalization(momentum=0.95)) # 根据batch_size修改
    model.add(ReLU())
    model.add(MaxPooling2D((3, 3),strides=1,padding='same'))
    
    model.add(Flatten())
    model.add(Dense(1024, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dropout(0.2))
    model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dropout(0.2))
    model.add(Dense(10, activation='softmax'))
    
    # ========================= 优化器 =======================================
    MODEL_OPTIMIZER = optimizers.SGD(lr=lr, momentum=0.9, nesterov=True)
    # MODEL_OPTIMIZER = optimizers.Adam()
    # 编译模型
    model.compile(loss=MODEL_LOSS, optimizer=MODEL_OPTIMIZER, metrics=[MODEL_METRIC])
    #model.summary()
    return model

In [9]:
InitialiazeModel(0.001)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 127, 127, 64)      640       
_________________________________________________________________
batch_normalization_9 (Batch (None, 127, 127, 64)      256       
_________________________________________________________________
re_lu_9 (ReLU)               (None, 127, 127, 64)      0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 127, 127, 64)      0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 63, 63, 64)        36928     
_________________________________________________________________
batch_normalization_10 (Batc (None, 63, 63, 64)        256       
_________________________________________________________________
re_lu_10 (ReLU)              (None, 63, 63, 64)       

## 训练

第一轮将momentum设为0.5，防止开始直接梯度爆炸

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping,TensorBoard,ReduceLROnPlateau
from math import ceil

train_path = "datasets\\Train"
val_path = "datasets\\Val_main"
BATCH_SIZE = 64
train_sample_num = 30000
val_sample_num = 1555

# 训练
model = InitialiazeModel(lr=0.02)
weights_path_name = "model_weight\\LBP\\lbp{epoch:02d}.hdf5" 
callbacks = [ModelCheckpoint(weights_path_name, monitor='val_loss', save_best_only=True, verbose=0,
                                             save_weights_only=True),
             EarlyStopping(monitor='val_loss', patience=5, verbose=0.001),
             TensorBoard(log_dir='train_log\\Alex',update_freq='epoch'),
             ReduceLROnPlateau(factor=0.2,
                               patience=2, 
                              min_lr=0.001)]
history1 = model.fit_generator(generator = LBP_gen(train_path,category),
                    validation_data = LBP_gen(val_path,category),
                    epochs = 100,
                    steps_per_epoch=ceil(train_sample_num/ BATCH_SIZE),
                    validation_steps=ceil(val_sample_num/ BATCH_SIZE),
                   max_queue_size=20,
                   callbacks=callbacks,
                   verbose = 1)

Instructions for updating:
Please use Model.fit, which supports generators.
Found 30150 images belonging to 10 classes.
Epoch 1/100
  1/469 [..............................] - ETA: 0s - loss: 4.6229 - categorical_accuracy: 0.1094WARNING:tensorflow:From F:\anaconda\envs\tensorflow2.3.1\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
469/469 [==============================] - 470s 1s/step - loss: 2.9027 - categorical_accuracy: 0.2556 - val_loss: 2.1808 - val_categorical_accuracy: 0.2379
Epoch 2/100
 20/469 [>.............................] - ETA: 6:33 - loss: 1.9606 - categorical_accuracy: 0.3036

第二轮开始设置momentum=0.9,继续训练

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping,TensorBoard,ReduceLROnPlateau
from math import ceil

train_path = "datasets\\Train"
val_path = "datasets\\Val_main"
BATCH_SIZE = 64
train_sample_num = 30000
val_sample_num = 1555

# 训练
model = InitialiazeModel(lr=0.02)
model.load_weights('model_weight/LBP/lbp01.hdf5')
weights_path_name = "model_weight\\LBP\\lbp{epoch:02d}+1.hdf5" 
callbacks = [ModelCheckpoint(weights_path_name, monitor='val_loss', save_best_only=True, verbose=0,
                                             save_weights_only=True),
             EarlyStopping(monitor='val_loss', patience=5, verbose=0.001),
             ReduceLROnPlateau(factor=0.2,
                               patience=2, 
                              min_lr=0.001)]
history1 = model.fit_generator(generator = LBP_gen(train_path,category),
                    validation_data = LBP_gen(val_path,category),
                    epochs = 100,
                    steps_per_epoch=ceil(train_sample_num/ BATCH_SIZE),
                    validation_steps=ceil(val_sample_num/ BATCH_SIZE),
                   max_queue_size=20,
                   callbacks=callbacks,
                   verbose = 1)

Instructions for updating:
Please use Model.fit, which supports generators.
Found 30150 images belonging to 10 classes.
Epoch 1/100
469/469 [==============================] - 477s 1s/step - loss: 1.7648 - categorical_accuracy: 0.3800 - val_loss: 2.1925 - val_categorical_accuracy: 0.3048
Epoch 2/100
469/469 [==============================] - 485s 1s/step - loss: 1.4883 - categorical_accuracy: 0.4789 - val_loss: 2.9381 - val_categorical_accuracy: 0.2244
Epoch 3/100
469/469 [==============================] - 483s 1s/step - loss: 1.3060 - categorical_accuracy: 0.5506 - val_loss: 1.7073 - val_categorical_accuracy: 0.4103
Epoch 4/100
469/469 [==============================] - 471s 1s/step - loss: 1.1491 - categorical_accuracy: 0.6062 - val_loss: 5.4318 - val_categorical_accuracy: 0.2334
Epoch 5/100
469/469 [==============================] - 471s 1s/step - loss: 1.0186 - categorical_accuracy: 0.6505 - val_loss: 5.0052 - val_categorical_accuracy: 0.2958
Epoch 6/100
 93/469 [====>..............